# 1. Importações

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# 2. Leitura dos Dados

In [3]:
path = '/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/01. Dados/02. Dados para treino e teste 50dim/'
x_train = pd.read_csv(path + 'x_train.csv', index_col=False).drop('Unnamed: 0', axis=1)
x_test = pd.read_csv(path + 'x_test.csv', index_col=False).drop('Unnamed: 0', axis=1)
y_train = pd.read_csv(path + 'y_train.csv', index_col=False).drop('Unnamed: 0', axis=1)
y_test = pd.read_csv(path + 'y_test.csv', index_col=False).drop('Unnamed: 0', axis=1)

len(x_train), len(x_test), len(y_train), len(y_test)

(1106, 277, 1106, 277)

In [4]:
x_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,1.142773,1.446013,-0.276407,0.711202,0.371399,1.674712,-0.652592,0.750868,-0.573086,0.068907,-0.041245,0.379596,0.591248,-0.485716,-0.121838,0.524532,-0.508232,0.215424,-0.649916,-0.243308,-0.179869,-0.207650,-1.073621,0.189484,0.559359,-0.123828,0.989256,0.749234,-0.408673,1.020104,0.194838,0.458985,-0.229972,-0.783636,1.554915,-0.373472,-0.693779,-0.225859,-0.967624,0.974142,-0.446093,1.543526,-0.253217,1.279842,1.002478,-1.669178,0.455464,0.224051,-0.743490,-1.485096
1,-0.110508,0.166793,0.127804,2.520468,1.382226,2.763531,-0.249695,2.423679,0.192702,-0.078511,0.452531,-0.387671,0.087219,0.589261,0.201948,0.855234,-1.095632,-1.936584,-1.532105,1.598647,0.179441,1.111156,-0.417048,1.623078,0.185025,0.224718,-0.471793,0.473032,0.731490,1.772383,2.865561,0.105851,0.908176,-0.947627,0.032539,0.852977,0.380596,0.980211,-0.775988,0.698551,0.845146,1.270311,-0.956371,3.494143,0.782756,0.440209,0.197112,-0.120969,-0.416663,0.905830
2,1.822024,0.926109,0.334602,0.577496,1.427653,1.211956,-0.241313,1.186324,1.666995,1.211566,-0.262563,0.328125,-1.061916,-0.792432,0.742074,0.854444,-0.702630,-0.831413,-2.346814,0.047639,-2.488856,2.001034,-1.114121,1.974115,0.263130,-0.900024,-0.595054,-0.194882,-0.673088,1.232286,1.226795,-1.028807,1.021789,-2.744594,1.656259,0.961233,-1.035030,-0.367986,-0.305127,1.626643,1.533291,0.819647,-1.451314,2.525155,-1.015526,-0.088621,0.080482,-0.066571,-1.459541,-1.520812
3,0.866334,0.463222,0.768105,0.097729,1.946827,1.341641,-0.910928,0.251836,0.735125,-1.360248,-0.486046,0.588617,-1.583984,0.058681,-0.639036,-1.281866,0.377621,-1.679610,-2.523722,0.703031,-1.121308,1.344754,-0.618831,0.574340,1.863863,-1.496150,1.302444,1.337962,0.635372,0.462527,0.559476,0.177885,1.468642,-1.275458,1.092704,-1.360870,-0.878649,-0.269013,-0.588579,0.718226,-0.555105,0.435449,-1.017609,0.938436,2.022405,0.373941,-0.596245,-1.007401,-1.369233,-1.365824
4,0.863443,0.154578,0.616606,0.611716,1.482627,0.932821,-0.649963,0.315175,1.684772,0.044644,-1.716206,2.032947,-1.425298,1.329936,-0.077829,1.597947,-1.087135,-1.286400,-2.036213,-0.027928,-3.427470,3.229930,-0.263983,2.187384,2.148592,-0.620556,0.463674,-0.470230,-0.970154,0.713380,0.989604,-0.273635,3.481610,-0.890182,1.606881,1.759937,0.289733,-0.292704,0.020724,1.983585,0.253344,1.436748,-0.376207,0.861265,0.824330,0.270567,-0.065160,0.332008,-1.706954,-0.414148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,1.878721,-0.609968,1.108881,0.712729,-0.690036,3.724357,-1.549477,2.444264,1.102822,-0.616896,0.630835,-0.648468,-0.488331,-0.907596,0.418230,0.199528,-0.692245,-1.250255,-0.521106,-0.634901,1.165634,-1.551132,-1.592144,-0.209284,-0.297352,0.495444,-0.535782,1.009038,1.030437,0.767096,3.016356,-0.972981,0.331405,0.439896,0.219986,-0.121540,-1.407367,1.496468,-1.438962,-0.327572,-0.181963,2.180817,-2.055367,2.222277,2.559773,-0.452650,0.633847,1.204797,0.087508,0.922227
1102,-0.145145,-0.489946,1.625201,1.188315,0.029281,4.702093,-1.545952,3.577367,-0.577639,1.077203,-1.445562,0.213037,-1.501619,0.610692,-0.130984,0.874045,-1.609659,-2.159281,-1.020277,1.950525,1.692230,0.545684,-2.332598,0.768902,-0.007309,0.912607,-0.187334,-0.263437,2.241750,2.133195,3.822438,0.109110,-0.022671,-0.385379,-1.624277,0.999325,-0.805288,1.970690,-1.834018,0.206801,0.192524,2.064622,-1.206330,3.214384,2.888214,0.052474,0.105301,1.235275,-0.189907,1.636873
1103,0.359816,-0.099839,0.550703,-0.389162,1.171448,-0.613196,-2.777513,0.403253,0.110069,2.141911,0.254180,2.323887,-1.858945,1.850336,-0.104664,1.825586,-0.839591,0.088525,-1.451757,-0.032757,-2.200614,1.171376,-1.817907,0.723297,2.130512,-0.901682,0.897581,0.776906,0.510441,-0.062833,-0.323624,-0.671026,2.310621,-1.411005,-1.28914

# 3. Regressão Logística

In [5]:
class LosgisticRegression():
    def __init__(self, alpha=0.001, epochs=100):
        self.alpha = alpha 
        self.epochs = epochs
        self.errors = []
        

    def sigmoide(self, x, w):
        return 1/(1+np.exp(-x @ w))

    def loss_cross_entropy(self, sum_error):
        return (1/(2*self.N))*sum_error

    def fit(self, x_train, y_train):
        self.D = x_train.shape[1]
        self.N = x_train.shape[0]
        self.w = np.zeros(self.D + 1)
        x_train = np.hstack((np.ones((self.N, 1)), x_train))

        for epoch in range(self.epochs):
            sum_error, sum_ = 0, 0
            for i in range(self.N):
                current_error = y_train[i] - self.sigmoide(x_train[i], self.w)
                sum_error += current_error**2
                sum_ += current_error * np.transpose(x_train[i])
            self.w = np.transpose(self.w) + (self.alpha * (1/self.N) * sum_)
            self.errors.append(self.loss_cross_entropy(sum_error))

    def predict(self, x_test):
        x_test = np.hstack((np.ones((x_test.shape[0], 1)), x_test))
        return [1 if self.sigmoide(row, self.w) >= 0.5 else 0 for row in x_test]
        

## Treino

In [6]:
num_epochs = 250
alphas = [0.01, 0.001, 0.0001, 0.00001, 5, 3, 10]

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_validation, Y_train, Y_validation = train_test_split(x_train, y_train["classification"], test_size=0.2, random_state=42)
len(X_train), len(X_validation), len(Y_train), len(Y_validation)

(884, 222, 884, 222)

In [8]:
print("------ GRID SEARCH ---------")
accs = []
for alpha in alphas:
    lg = LosgisticRegression(alpha, num_epochs)
    lg.fit(X_train, Y_train.values)
    y_predict = lg.predict(X_validation)
    print(" ------------ ALPHA = " + str(alpha) + " --------------------")
    print('Test Accuraccy: ', accuracy_score(Y_validation, y_predict))
    print('\nConfusion Matrix:')
    print(confusion_matrix(Y_validation, y_predict))
    print(classification_report(Y_validation, y_predict, output_dict=True))
    accs.append((alpha, accuracy_score(Y_validation, y_predict)))
    report = classification_report(Y_validation, y_predict, output_dict=True)



------ GRID SEARCH ---------
 ------------ ALPHA = 0.01 --------------------
Test Accuraccy:  0.7522522522522522

Confusion Matrix:
[[97 14]
 [41 70]]
{'0': {'precision': 0.7028985507246377, 'recall': 0.8738738738738738, 'f1-score': 0.7791164658634537, 'support': 111}, '1': {'precision': 0.8333333333333334, 'recall': 0.6306306306306306, 'f1-score': 0.7179487179487181, 'support': 111}, 'accuracy': 0.7522522522522522, 'macro avg': {'precision': 0.7681159420289856, 'recall': 0.7522522522522522, 'f1-score': 0.7485325919060859, 'support': 222}, 'weighted avg': {'precision': 0.7681159420289855, 'recall': 0.7522522522522522, 'f1-score': 0.7485325919060859, 'support': 222}}
 ------------ ALPHA = 0.001 --------------------
Test Accuraccy:  0.5945945945945946

Confusion Matrix:
[[110   1]
 [ 89  22]]
{'0': {'precision': 0.5527638190954773, 'recall': 0.990990990990991, 'f1-score': 0.7096774193548386, 'support': 111}, '1': {'precision': 0.9565217391304348, 'recall': 0.1981981981981982, 'f1-score':

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':


 ------------ ALPHA = 10 --------------------
Test Accuraccy:  0.7297297297297297

Confusion Matrix:
[[ 57  54]
 [  6 105]]
{'0': {'precision': 0.9047619047619048, 'recall': 0.5135135135135135, 'f1-score': 0.6551724137931034, 'support': 111}, '1': {'precision': 0.660377358490566, 'recall': 0.9459459459459459, 'f1-score': 0.7777777777777778, 'support': 111}, 'accuracy': 0.7297297297297297, 'macro avg': {'precision': 0.7825696316262354, 'recall': 0.7297297297297297, 'f1-score': 0.7164750957854407, 'support': 222}, 'weighted avg': {'precision': 0.7825696316262354, 'recall': 0.7297297297297297, 'f1-score': 0.7164750957854406, 'support': 222}}


In [9]:
lista_ = [acc for alpha, acc in accs]
best_alpha = accs[lista_.index(max(lista_))][0]
best_alpha

3

In [10]:
lg = LosgisticRegression(best_alpha, num_epochs)
lg.fit(x_train, y_train['classification'].values)

## Predição

In [11]:
y_predict = lg.predict(x_test)

## Métricas

In [12]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)

Test Accuraccy:  0.7292418772563177

Confusion Matrix:
[[119  11]
 [ 64  83]]


In [13]:
report

{'0': {'f1-score': 0.7603833865814696,
  'precision': 0.6502732240437158,
  'recall': 0.9153846153846154,
  'support': 130},
 '1': {'f1-score': 0.6887966804979254,
  'precision': 0.8829787234042553,
  'recall': 0.564625850340136,
  'support': 147},
 'accuracy': 0.7292418772563177,
 'macro avg': {'f1-score': 0.7245900335396975,
  'precision': 0.7666259737239856,
  'recall': 0.7400052328623756,
  'support': 277},
 'weighted avg': {'f1-score': 0.7223933295624045,
  'precision': 0.7737667561953379,
  'recall': 0.7292418772563177,
  'support': 277}}

## Salvando resultados

In [14]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
    'hiperparams': str({'alpha': best_alpha})
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/1. Regressão Logística/regressao_logistica_50dim.json')

In [15]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/1. Regressão Logística/regressao_logistica_50dim.json')

,precision,recall,f1score,y_predict,y_real,hiperparams
0,0.773767,0.729242,0.722393,"[0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...",{'alpha': 3}


# 4. Análise Discriminante Gaussiano

In [16]:
import numpy as np
from scipy.stats import multivariate_normal

class GaussianDiscriminant():
  def __init__(self):
    pass

  def fitAGD(self, X, y):
    self.__classes = np.unique(y) # pegando as classes e ocorrencias
    numClasses = len(self.__classes) #numero de classes
    n, numFeatures = X.shape # numero de linhas e colunas do dataset 
    
    self.__sigma = 0
    self.__probabilidadeClasses = np.zeros((numClasses, 1))
    self.__media = np.zeros((numClasses, numFeatures)) # criando a lista da media das classes por feature

    for i in range(numClasses):
        # flatnonzero retorna os indices em que os valores são diferentes de zero. O retorno é uma lista flat. 
        # A comparação y == classes vai retornar um array de 0's e 1's. 
        # E portanto a operaçao final vai pegar todos os indices em que a classe é igual ao y.
        indices = np.flatnonzero(y == self.__classes[i])

        self.__probabilidadeClasses[i] = len(indices)/ n
        self.__media[i] = np.mean(X[indices], axis=0)
        self.__sigma += np.cov(X[indices].T) *(len(indices)-1)
        
    self.__sigma /= n
    
    return {'media': self.__media, 'covar': self.__sigma, 'classes': self.__classes, 'numRows': n, 'numClasses': numClasses, 
            'numFeatures': numFeatures, 'probabilidadeClasses': self.__probabilidadeClasses }

  def predict(self,X):
    # Usando a distribuição gaussiana multivariada da biblioteca scipy.stats
    # Basicamente o calculo da função densidade de probabilidade visto na imagem abaixo.
    pdf = lambda mean: multivariate_normal.pdf(X, mean=mean, cov=self.__sigma)

    # Apply a function to 1-D slices along the given axis
    # No caso aplica a função fdp no axis 1 das médias
    y_probs = np.apply_along_axis(pdf, 1, self.__media) * self.__probabilidadeClasses

    return self.__classes[np.argmax(y_probs, axis=0)]


## Treino

In [17]:
agd = GaussianDiscriminant()
fit = agd.fitAGD(x_train.to_numpy(), y_train)


## Predição

In [18]:
y_predict = agd.predict(x_test.to_numpy())

## Métricas

In [19]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)

Test Accuraccy:  0.8086642599277978

Confusion Matrix:
[[103  27]
 [ 26 121]]


In [20]:
report

{'0': {'f1-score': 0.7953667953667953,
  'precision': 0.7984496124031008,
  'recall': 0.7923076923076923,
  'support': 130},
 '1': {'f1-score': 0.8203389830508475,
  'precision': 0.8175675675675675,
  'recall': 0.8231292517006803,
  'support': 147},
 'accuracy': 0.8086642599277978,
 'macro avg': {'f1-score': 0.8078528892088215,
  'precision': 0.8080085899853342,
  'recall': 0.8077184720041863,
  'support': 277},
 'weighted avg': {'f1-score': 0.8086191837767436,
  'precision': 0.8085952420391175,
  'recall': 0.8086642599277978,
  'support': 277}}

## Salvando resultados

In [21]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/3. Análise Discriminante Gaussiano/agd_50dim.json')

In [22]:
resultados

,precision,recall,f1score,y_predict,y_real
0,0.808595,0.808664,0.808619,"[1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ..."


In [23]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/3. Análise Discriminante Gaussiano/agd_50dim.json')

,precision,recall,f1score,y_predict,y_real
0,0.808595,0.808664,0.808619,"[1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ..."


# 5. Árvore de Decisão

## Treino

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

max_depths = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
params = {'criterion': ['gini','entropy'], 'max_depth': max_depths}

clf = GridSearchCV(DecisionTreeClassifier(), params, verbose=2)
    
print("[Árvore de Decisão] Treinando modelos...")
clf.fit(x_train, y_train['classification'].values)

[Árvore de Decisão] Treinando modelos...
Fitting 5 folds for each of 26 candidates, totalling 130 fits
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.0s
[CV] criterion=gini, max_depth=5 ............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ..................... criterion=gini, max_depth=10, total=   0.0s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.0s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.0s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.0s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.0s
[CV] criterion=gini, max_depth=15 ....................................
[CV] ..................... criterion=gini, max_depth=15, total=   0.0s
[CV] criterion=gini, max_depth=15 ....................................
[CV] ..................... criterion=gini, max_depth=15, total=   0.0s
[CV] criterion=gini, max_depth=15 ....................................
[CV] .

[Parallel(n_jobs=1)]: Done 130 out of 130 | elapsed:    7.1s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                

In [25]:
params = clf.best_params_
print("Parâmetros escolhidos para Árvore de Decisão: ", clf.best_params_)

Parâmetros escolhidos para Árvore de Decisão:  {'criterion': 'entropy', 'max_depth': 10}


## Predição

In [26]:
y_predict = clf.predict(x_test)

## Métricas

In [27]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)
report

Test Accuraccy:  0.6895306859205776

Confusion Matrix:
[[ 82  48]
 [ 38 109]]


{'0': {'f1-score': 0.6559999999999999,
  'precision': 0.6833333333333333,
  'recall': 0.6307692307692307,
  'support': 130},
 '1': {'f1-score': 0.7171052631578948,
  'precision': 0.6942675159235668,
  'recall': 0.7414965986394558,
  'support': 147},
 'accuracy': 0.6895306859205776,
 'macro avg': {'f1-score': 0.6865526315789474,
  'precision': 0.68880042462845,
  'recall': 0.6861329147043433,
  'support': 277},
 'weighted avg': {'f1-score': 0.6884277028310849,
  'precision': 0.6891359500869951,
  'recall': 0.6895306859205776,
  'support': 277}}

## Salvando resultados

In [28]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
    'hiperparams': str(clf.best_params_)
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/5. Árvore de Decisão/ad_50dim.json')

In [29]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/5. Árvore de Decisão/ad_50dim.json')

,precision,recall,f1score,y_predict,y_real,hiperparams
0,0.689136,0.689531,0.688428,"[1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...","{'criterion': 'entropy', 'max_depth': 10}"


# 6. Random Forest

## Treino

In [30]:
from sklearn.ensemble import RandomForestClassifier

print("[RANDOM FOREST] Selecionando hiperparâmetros...")
num_estimators = [100, 110, 120, 130, 140, 150]
max_depths = [1, 5, 10, 15, 20, 25, 55, 60]

params = {'criterion': ['gini','entropy'],
          'n_estimators': num_estimators,
          'max_depth': max_depths
          }

# params
clt = GridSearchCV(RandomForestClassifier(), params, verbose=2)

print("[RANDOM FOREST] Treinando modelo...")
clf.fit(x_train, y_train['classification'])

[RANDOM FOREST] Selecionando hiperparâmetros...
[RANDOM FOREST] Treinando modelo...
Fitting 5 folds for each of 26 candidates, totalling 130 fits
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=1 .....................................
[CV] ...................... criterion=gini, max_depth=1, total=   0.0s
[CV] criterion=gini, max_depth=5 .....................................
[CV] ...................... criterion=gini, max_depth=5, total=   0.0s
[C

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ..................... criterion=gini, max_depth=10, total=   0.0s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.0s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.0s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.0s
[CV] criterion=gini, max_depth=10 ....................................
[CV] ..................... criterion=gini, max_depth=10, total=   0.0s
[CV] criterion=gini, max_depth=15 ....................................
[CV] ..................... criterion=gini, max_depth=15, total=   0.0s
[CV] criterion=gini, max_depth=15 ....................................
[CV] ..................... criterion=gini, max_depth=15, total=   0.0s
[CV] criterion=gini, max_depth=15 ....................................
[CV] .

[Parallel(n_jobs=1)]: Done 130 out of 130 | elapsed:    7.2s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                

In [31]:
print("[RANDOM FOREST] Hiperparâmetros escolhidos para Radom Forest: ", clf.best_params_)

[RANDOM FOREST] Hiperparâmetros escolhidos para Radom Forest:  {'criterion': 'entropy', 'max_depth': 10}


## Predição

In [32]:
y_predict = clf.predict(x_test)

## Métricas

In [33]:
print('Test Accuraccy: ', accuracy_score(y_test, y_predict))
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_predict))
report = classification_report(y_test, y_predict, output_dict=True)
report

Test Accuraccy:  0.6859205776173285

Confusion Matrix:
[[ 82  48]
 [ 39 108]]


{'0': {'f1-score': 0.6533864541832669,
  'precision': 0.6776859504132231,
  'recall': 0.6307692307692307,
  'support': 130},
 '1': {'f1-score': 0.7128712871287128,
  'precision': 0.6923076923076923,
  'recall': 0.7346938775510204,
  'support': 147},
 'accuracy': 0.6859205776173285,
 'macro avg': {'f1-score': 0.6831288706559899,
  'precision': 0.6849968213604577,
  'recall': 0.6827315541601255,
  'support': 277},
 'weighted avg': {'f1-score': 0.6849542175153267,
  'precision': 0.6854455029709378,
  'recall': 0.6859205776173285,
  'support': 277}}

## Salvando métricas

In [34]:
resultados = pd.DataFrame({
    'precision': report['weighted avg']['precision'], 
    'recall': report['weighted avg']['recall'], 
    'f1score': report['weighted avg']['f1-score'], 
    'y_predict': [y_predict],
    'y_real': [y_test['classification'].values],
    'hiperparams': str(clf.best_params_)
})
resultados.to_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/8. Random Forest/random_forest_50dim.json')

In [35]:
pd.read_json('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/05. Resultados/5.1. Resultados dos modelos/8. Random Forest/random_forest_50dim.json')

,precision,recall,f1score,y_predict,y_real,hiperparams
0,0.685446,0.685921,0.684954,"[1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, ...","{'criterion': 'entropy', 'max_depth': 10}"
